https://github.com/SpaceNetChallenge/SpaceNet_Off_Nadir_Solutions/blob/master/selim_sef/training/losses.py

In [42]:
%autosave 60 

Autosaving every 60 seconds


In [43]:
#default_exp segmentation.losses_binary

In [44]:
import fastai; print(fastai.__version__)

1.0.58.dev0


In [45]:
#export
from fastai.vision import *

In [78]:
# export
cross_entropy = CrossEntropyFlat(axis=1)

### test

In [46]:
from local.test import *

In [47]:
_input = torch.ones((1,2,3,3))

_input2 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*1e3

_input3 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*-1e3

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input = torch.cat([_input, _input])
_input2 = torch.cat([_input2, _input2])
_input3 = torch.cat([_input3, _input3])
_target = torch.cat([_target, _target])

_input.size(), _target.size()

(torch.Size([2, 2, 3, 3]), torch.Size([2, 1, 3, 3]))

In [48]:
#export
class _SingleSoftmaxOutput(nn.Module):
    "layer for [B,2,H,W] logits -> [B,1,H,W] probas"
    def __init__(self):
        super().__init__()
    def forward(self, input, output):
        input = input.softmax(1)[:,1].unsqueeze(1).contiguous()
        return input, output

_single_softmax_layer = _SingleSoftmaxOutput()

### `cross_entropy`

In [79]:
cross_entropy(_input, _target)

tensor(0.6931)

In [80]:
cross_entropy(_input2, _target)

tensor(0.)

In [81]:
cross_entropy(_input3, _target)

tensor(1000.)

### `dice_loss_v2`

In [49]:
# export
def _soft_dice_loss(outputs, targets, per_image=False):    
    batch_size = outputs.size()[0]
    eps = 1e-5
    if not per_image: batch_size = 1
    dice_target = targets.contiguous().view(batch_size, -1).float()
    dice_output = outputs.contiguous().view(batch_size, -1)
    intersection = torch.sum(dice_output * dice_target, dim=1)
    union = torch.sum(dice_output, dim=1) + torch.sum(dice_target, dim=1) + eps
    loss = (1 - (2 * intersection + eps) / union).mean()
    return loss

class _DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, per_image=False):
        super().__init__()
        self.size_average = size_average
        self.register_buffer('weight', weight)
        self.per_image = per_image

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        return _soft_dice_loss(input, target, per_image=self.per_image)


dice_loss_v2 = _DiceLoss()

In [50]:
dice_loss_v2(_input, _target)

tensor(0.5294)

In [51]:
dice_loss_v2(_input2, _target)

tensor(0.)

In [52]:
dice_loss_v2(_input3, _target)

tensor(1.0000)

### `stable_bce_loss`

In [53]:
#export
class _StableBCELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        input = input.float().view(-1)
        target = target.float().view(-1)
        neg_abs = - input.abs()
        # todo check correctness
        loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
        return loss.mean()
    
stable_bce_loss = _StableBCELoss()

In [54]:
stable_bce_loss(_input, _target)

tensor(0.7519)

In [55]:
stable_bce_loss(_input2, _target)

tensor(0.5243)

In [56]:
stable_bce_loss(_input3, _target)

tensor(1.0377)

### `jaccard_loss`

In [57]:
#export
def _jaccard(outputs, targets, per_image=False, non_empty=False, min_pixels=5):
    batch_size = outputs.size()[0]
    eps = 1e-3
    if not per_image:
        batch_size = 1
    dice_target = targets.contiguous().view(batch_size, -1).float()
    dice_output = outputs.contiguous().view(batch_size, -1)
    target_sum = torch.sum(dice_target, dim=1)
    intersection = torch.sum(dice_output * dice_target, dim=1)
    losses = 1 - (intersection + eps) / (torch.sum(dice_output + dice_target, dim=1) - intersection + eps)
    if non_empty:
        assert per_image == True
        non_empty_images = 0
        sum_loss = 0
        for i in range(batch_size):
            if target_sum[i] > min_pixels:
                sum_loss += losses[i]
                non_empty_images += 1
        if non_empty_images == 0:
            return 0
        else:
            return sum_loss / non_empty_images

    return losses.mean()

class _JaccardLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, per_image=False, 
                 non_empty=False, apply_sigmoid=False, min_pixels=5):
        super().__init__()
        self.size_average = size_average
        self.register_buffer('weight', weight)
        self.per_image = per_image
        self.non_empty = non_empty
        self.apply_sigmoid = apply_sigmoid
        self.min_pixels = min_pixels

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        return _jaccard(input, target, per_image=self.per_image, non_empty=self.non_empty, min_pixels=self.min_pixels)
    
jaccard_loss = _JaccardLoss()

In [58]:
jaccard_loss (_input, _target)

tensor(0.6923)

In [59]:
jaccard_loss (_input2, _target)

tensor(0.)

In [60]:
jaccard_loss (_input3, _target)

tensor(0.9999)

### `lovasz_loss`

In [61]:
#export
def _lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts.float() - gt_sorted.float().cumsum(0)
    union = gts.float() + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1:  # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


def _lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = _mean(_lovasz_hinge_flat(*_flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                    for log, lab in zip(logits, labels))
    else:
        loss =_lovasz_hinge_flat(*_flatten_binary_scores(logits, labels, ignore))
    return loss


def _lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * (signs))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad =_lovasz_grad(gt_sorted)
    loss = torch.dot(F.relu(errors_sorted), (grad))
    return loss


def _flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels


def _lovasz_sigmoid(probas, labels, per_image=False, ignore=None):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
    """
    if per_image:
        loss = _mean(_lovasz_sigmoid_flat(*_flatten_binary_scores(prob.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for prob, lab in zip(probas, labels))
    else:
        loss =_lovasz_sigmoid_flat(*_flatten_binary_scores(probas, labels, ignore))
    return loss


def _lovasz_sigmoid_flat(probas, labels):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      only_present: average only on classes present in ground truth
    """
    fg = labels.float()
    errors = ((fg) - probas).abs()
    errors_sorted, perm = torch.sort(errors, 0, descending=True)
    perm = perm.data
    fg_sorted = fg[perm]
    loss = torch.dot(errors_sorted, (_lovasz_grad(fg_sorted)))
    return loss

def _symmetric_lovasz(outputs, targets, ):
    return (_lovasz_hinge(outputs, targets) +_lovasz_hinge(-outputs, 1 - targets)) / 2

def _mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n

class _LovaszLoss(nn.Module):
    def __init__(self, ignore_index=0, per_image=True):
        super().__init__()
        self.ignore_index = ignore_index
        self.per_image = per_image

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        input = input.contiguous()
        target = target.contiguous()
        return _symmetric_lovasz(input, target)
    
lovasz_loss = _LovaszLoss()

In [62]:
lovasz_loss(_input, _target)

tensor(1.2778)

In [63]:
lovasz_loss(_input2, _target)

tensor(0.7778)

In [64]:
lovasz_loss(_input3, _target)

tensor(1.7778)

### `lovasz_loss_sigmoid`

In [65]:
#export
class _LovaszLossSigmoid(nn.Module):
    def __init__(self, ignore_index=0, per_image=True):
        super().__init__()
        self.ignore_index = ignore_index
        self.per_image = per_image

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        input = input.contiguous()
        target = target.contiguous()
        return _lovasz_sigmoid(input, target, per_image=self.per_image, ignore=self.ignore_index)

lovasz_loss_sigmoid = _LovaszLossSigmoid()    

In [66]:
lovasz_loss_sigmoid(_input, _target)

tensor(0.5000)

In [67]:
lovasz_loss_sigmoid(_input2, _target)

tensor(0.)

In [68]:
lovasz_loss_sigmoid(_input3, _target)

tensor(1.)

### `focal_loss`

In [69]:
#export
class _FocalLoss2d(nn.Module):
    def __init__(self, gamma=2, ignore_index=255):
        super().__init__()
        self.gamma = gamma
        self.ignore_index = ignore_index

    def forward(self, input, target):
        input, target = _single_softmax_layer(input, target)
        input = input.contiguous()
        target = target.contiguous()
        eps = 1e-8
        non_ignored = target.view(-1) != self.ignore_index
        target = target.view(-1)[non_ignored].float()
        input = input.contiguous().view(-1)[non_ignored]
        input = torch.clamp(input, eps, 1. - eps)
        target = torch.clamp(target, eps, 1. - eps)
        pt = (1 - target) * (1 - input) + target * input
        return (-(1. - pt) ** self.gamma * torch.log(pt)).mean()
    
focal_loss = _FocalLoss2d()

In [70]:
focal_loss(_input, _target)

tensor(0.1733)

In [71]:
focal_loss(_input2, _target)

tensor(0.)

In [72]:
focal_loss(_input3, _target)

tensor(18.4207)

### `combo_loss`

In [73]:
#export
class _ComboLoss(nn.Module):
    def __init__(self, weights, per_image=False):
        super().__init__()
        self.weights = weights
        self.bce = _StableBCELoss()
        self.dice = _DiceLoss(per_image=False)
        self.jaccard = _JaccardLoss(per_image=False)
        self.lovasz = _LovaszLoss(per_image=per_image)
        self.lovasz_sigmoid = _LovaszLossSigmoid(per_image=per_image)
        self.focal = _FocalLoss2d()
        self.losses = {'bce': self.bce,
                        'dice': self.dice,
                        'focal': self.focal,
                        'jaccard': self.jaccard,
                        'lovasz': self.lovasz,
                        'lovasz_sigmoid': self.lovasz_sigmoid}

    def forward(self, input, target):
        loss = 0
        for k, w in self.weights.items(): loss += w * self.losses[k](input, target)
        return loss.clamp(min=1e-5)

_weights = {'bce': 1, 'dice': 1, 'focal': 1, 'jaccard': 1, 'lovasz': 1, 'lovasz_sigmoid': 1}
combo_loss = _ComboLoss(_weights)

In [74]:
combo_loss(_input, _target)

tensor(3.9246)

In [75]:
combo_loss(_input2, _target)

tensor(1.3021)

In [76]:
combo_loss(_input3, _target)

tensor(24.2361)

### export

In [82]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.
